In [1]:
import torch
import torch.nn as nn
import numpy as np
import csv

import import_ipynb

from tools.ToolBox import json_loader
from tools.ToolBox import csv_saver

from tools.CNN_Tools_copy import VGG_Schmidtea
from tools.CNN_Tools_copy import train
from tools.CNN_Tools_copy import validate

from tools.Dataset_Tools import centriole_dataset
from tools.Dataset_Tools import dataset_creator
from tools.Dataset_Tools import dataset_loader
from tools.Dataset_Tools import dataset_aggregator

importing Jupyter notebook from /home/cyril_b/projects/Planarians/tools/ToolBox.ipynb
importing Jupyter notebook from /home/cyril_b/projects/Planarians/tools/CNN_Tools_copy.ipynb
importing Jupyter notebook from /home/cyril_b/projects/Planarians/tools/Dataset_Tools.ipynb


In [2]:
problem = 'regression'

n_class = 2

In [3]:
load_dataset = True

# Creation/Loading of the Dataset
if load_dataset == False:   
    train_loader, validation_loader = dataset_creator(path_json = './data_json/',
                                                     batch_size = 2,
                                                     n_class = n_class,
                                                     save_dataset = True)  
    
else: 
    train_loader, validation_loader = dataset_loader(path = './data/',
                                                 #train_set = 'train_data_pregression_b700_unNormalized.pth',
                                                 train_set = 'data_train_regression_normalized.pth',
                                                 val_set = 'validation_data_pregression_b700_unNormalized.pth')


In [4]:
load_weight = False


if torch.cuda.is_available():                                  
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print('Using PyTorch version:', torch.__version__, ' Device:', device)


# Criterion and CNN loading
criterion = nn.MSELoss()
model = VGG_Schmidtea(n_classes = 2).to(device)


# Weight loading
if load_weight == True:
    #model.load_state_dict(torch.load('./weight/VGG_schmidtea_weight_noReLu_80percent_classification_n72.pth'))
    model.load_state_dict(torch.load('./weight/VGG_schmidtea_weight_tmp.pth'))



# Optimizer loading
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.)


Using PyTorch version: 1.6.0  Device: cuda


In [5]:
def train_(model,train_loader, device, problem, criterion, optimizer, epoch, n_class, loss_vector, accuracy_vector):
    # Set model to training mode
    model.train()

    train_loss, correct = 0, 0
        
    # Loop over each batch from the training set
    for batch_idx, batch in enumerate(train_loader):
        
        # Send data to the device (GPU) in the appropriate format
        img = batch['image'].float().to(device)
        angle = batch['angle'].float().to(device)        

        # Zero gradient buffers
        optimizer.zero_grad()
        
        # Pass data through the network
        output = model(img)
        
        cosangle = torch.cos(torch.deg2rad(angle))
        sinangle = torch.sin(torch.deg2rad(angle))
        tangle = torch.stack([sinangle, cosangle]).view((len(angle), 2))
        
        loss = criterion(output, tangle)

        train_loss += loss
  
        # Backpropagate
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        ## Calcul of correct predicted angle
        real_angle = torch.rad2deg(torch.atan2(tangle[:,1], tangle[:,0]))
        pred_angle = torch.rad2deg(torch.atan2(output[:,1], output[:,0]))

        angle_5 = real_angle//5
        pred_5 = pred_angle//5
        correct += pred_5.eq(angle_5).cpu().sum()
            
    
    train_loss /= len(train_loader)
    loss_vector.append(train_loss.detach().cpu().numpy())
    
    accuracy = correct.to(torch.float32)/len(train_loader.dataset)*100
    accuracy_vector.append(accuracy.cpu().numpy())
    
    print(f" Training: Loss: {train_loss:.3f}, Accuracy: {accuracy:.1f}    |    ", end = '')
    
    
    
def validate_(model, validation_loader, device, problem, criterion, n_class, loss_vector, accuracy_vector):
    '''
    Input of the function:
        model: neural network model in Pytorch
        loss_vector: empty array with is assigned by the function
        accuracy_vector: empty array with is assigned by the function
    '''

    val_loss, correct = 0, 0
        
    i = 0
    for batch_idx, batch in enumerate(validation_loader):
        i +=1
        # Copy data to GPU if needed
        img = batch['image'].float().to(device)
        angle = batch['angle'].float().to(device)        
        
        # Pass data through the network
        with torch.no_grad():
                    
            # Pass data through the network
            output = model(img)

            cosangle = torch.cos(torch.deg2rad(angle))
            sinangle = torch.sin(torch.deg2rad(angle))
            tangle = torch.stack([sinangle, cosangle]).view((len(angle), 2))

            loss = criterion(output, tangle)

            val_loss += loss
    
            real_angle = torch.rad2deg(torch.atan2(tangle[:,1], tangle[:,0]))
            pred_angle = torch.rad2deg(torch.atan2(output[:,1], output[:,0]))
            
            angle_5 = real_angle//5
            pred_5 = pred_angle//5
            correct += pred_5.eq(angle_5).cpu().sum()
            
            #angle_list.append(real_angle.detach().flatten().numpy())
            #pred_list.append(real_angle.detach().flatten().numpy())
            
    
    val_loss /= len(validation_loader)
    loss_vector.append(val_loss.detach().cpu().numpy())
    
    accuracy = correct.to(torch.float32)/len(validation_loader.dataset)*100
    accuracy_vector.append(accuracy.detach().cpu().numpy())
    
    print(f" Validation: Loss: {val_loss:.3f}, Accuracy: {accuracy:.1f}")
    

In [7]:
%%time 

for epoch in range(10):
    losst, lossv = [], []
    accut, accuv = [], []
    
    
    print(f"Epoch {epoch}", end = '')
    train_(model, train_loader, device, problem, criterion, optimizer, epoch, n_class, losst, accut)
    validate_(model, validation_loader, device, problem, criterion, n_class, lossv, accuv)


Epoch 0 Training: Loss: 0.509, Accuracy: 1.4    |     Validation: Loss: 0.512, Accuracy: 1.4
Epoch 1 Training: Loss: 0.507, Accuracy: 1.4    |     Validation: Loss: 0.510, Accuracy: 1.3
Epoch 2 Training: Loss: 0.506, Accuracy: 1.4    |     Validation: Loss: 0.509, Accuracy: 1.4
Epoch 3 Training: Loss: 0.505, Accuracy: 1.4    |     Validation: Loss: 0.506, Accuracy: 1.5
Epoch 4 Training: Loss: 0.504, Accuracy: 1.4    |     Validation: Loss: 0.506, Accuracy: 1.5
Epoch 5 Training: Loss: 0.504, Accuracy: 1.4    |     Validation: Loss: 0.506, Accuracy: 1.4
Epoch 6 Training: Loss: 0.504, Accuracy: 1.4    |     Validation: Loss: 0.507, Accuracy: 1.4
Epoch 7 Training: Loss: 0.503, Accuracy: 1.4    |     Validation: Loss: 0.505, Accuracy: 1.5
Epoch 8 Training: Loss: 0.503, Accuracy: 1.4    |     Validation: Loss: 0.504, Accuracy: 1.5
Epoch 9 Training: Loss: 0.503, Accuracy: 1.4    |     Validation: Loss: 0.505, Accuracy: 1.4
CPU times: user 44min 30s, sys: 7.01 s, total: 44min 37s
Wall time: 29

In [11]:
print(losst[0])
lossv
accut

0.5078406


[array(1.3975164, dtype=float32)]

In [8]:
train_loader, validation_loader = dataset_loader(path = './data/',
                                                 #train_set = 'train_data_pregression_b700_unNormalized.pth',
                                                 train_set = 'data_train_regression_normalized.pth',
                                                 val_set = 'validation_data_pregression_b700_unNormalized.pth')


for idx, batch in enumerate(train_loader):
    if idx == 1:
        #print(batch['image'][1])
        toto = {'image': batch['image'][:2], 'angle':batch['angle'][:2]}
        tata = {'image': batch['image'][:4], 'angle':batch['angle'][:4]}
        
training_ = centriole_dataset(img_db = toto['image'], angle_db = toto['angle'], problem = problem)
testing_ = centriole_dataset(img_db = tata['image'], angle_db = tata['angle'], problem = problem)

train_loader_ = torch.utils.data.DataLoader(training_, batch_size = 2, shuffle = False, drop_last=True)
validation_loader_ = torch.utils.data.DataLoader(testing_, batch_size = 2, shuffle = False, drop_last=True)

In [21]:
model.to('cpu')
def turn2angl(data):
    return(torch.rad2deg(torch.atan2(data[:,1], data[:,0])))

for idx, batch in enumerate(validation_loader_):

    with torch.no_grad():
        model.eval()
        output = model(batch['image'].float().to('cpu'))
        angle = batch['angle'].to('cpu')
        
        cosangle = torch.cos(torch.deg2rad(angle))
        sinangle = torch.sin(torch.deg2rad(angle))
        tangle = torch.stack([sinangle, cosangle]).view((len(angle), 2))
        
        loss = criterion(output, tangle)
        
        print(output.numpy())
        print(tangle.numpy())
        
        print(turn2angl(output))
        print(turn2angl(tangle))
        
        print(loss)
        
        print('\n')
        


[[ 0.00011239 -0.00674243]
 [ 0.00023574 -0.00758078]]
[[-0.05704826  0.70710678]
 [ 0.99837142  0.70710678]]
tensor([-89.0450, -88.2188])
tensor([94.6125, 35.3084], dtype=torch.float64)
tensor(0.5050, dtype=torch.float64)


[[-0.00018176 -0.0063397 ]
 [-0.00087792 -0.0050795 ]]
[[-0.75925313 -0.06237769]
 [ 0.65079542 -0.99805262]]
tensor([-91.6422, -99.8059])
tensor([-175.3033,  -56.8930], dtype=torch.float64)
tensor(0.4975, dtype=torch.float64)




In [ ]:
%%time 

for epoch in range(1):
    losst, lossv = [], []
    accut, accuv = [], []
    
    
    print(f"Epoch {epoch}", end = '')
    train_(model, train_loader_, device, problem, criterion, optimizer, epoch, n_class, losst, accut)
    validate_(model, validation_loader, device, problem, criterion, n_class, lossv, accuv)